In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

2023-11-22 18:37:47.457445: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 18:37:47.457526: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 18:37:47.457572: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 18:37:48.707884: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 58.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pathlib
# Defina una función para leer los datos de texto y devolver pares de texto y etiqueta
def read_text_data(data_path):
    texts = []
    labels = []
    label_mapping = {'sis': 0, 'dis': 1, 'cic': 2, 'tis': 3}

    for label in label_mapping.keys():
        label_path = os.path.join(data_path, label)
        for text_file in os.listdir(label_path):
            with open(os.path.join(label_path, text_file), 'r', encoding='utf-8') as f:
                text = f.read()
            labels.append(label_mapping[label])
            texts.append(text)

    return texts, labels

In [ ]:
# Ruta de acceso al directorio del dataset
data_path = pathlib.Path("/content/drive/MyDrive/IA22023/DATASETS/NLPDatasetIng")
print(data_path)

# Leer los datos de texto y las etiquetas del directorio train
texts, labels = read_text_data(data_path/'train')

print(f'Successfully read {len(texts)} texts, and {len(labels)} labels from training dataset')

/content/drive/MyDrive/IA22023/DATASETS/NLPDatasetIng
Successfully read 3332 texts, and 3332 labels from training dataset


#### Processing the dataset with a text tokenizer and constrcut the vocabulary


We will use the `get_tokenizer()` function in the torchtext library from
`torchtext.data.utils`. This function can be used to create a tokenizer
that will be used to preprocess the text data, it takes an argument that
specifies the type of tokenizer to create, we will use in this case a
`basic_english` tokenizer. This is a simple type of tokenizer that
splits the text into words based on whitespace and punctuation marks,
converts all words to lowercase (i.e. standardizing and tokenizing).

In [ ]:
# from torchtext.data.utils import get_tokenizer

# # Define a tokenizer function to preprocess the text
# tokenizer = get_tokenizer('basic_english')

In [ ]:
import spacy
from torchtext.data.utils import get_tokenizer

# Cargar el modelo en español
nlp = spacy.load('es_core_news_sm')

# Definir una función de tokenización que utiliza el modelo de spacy
def tokenize(text):
    return [token.text for token in nlp(text)]

# Crear el tokenizador
tokenizer = get_tokenizer(tokenize)

Let’s see a sample standardized and tokenized text:

In [ ]:
tokenizer('Este es un ejemplo de texto en español.')

['Este', 'es', 'un', 'ejemplo', 'de', 'texto', 'en', 'español', '.']

Next, we’ll define a way to numercalize the tokens that can be created
from the previous tokenizer, in particular, we’ll index the tokens and
map them to the vocabulary constructed for the entire words in the text
corpus (i.e. indexing).

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

# Build the vocabulary from the text data
vocab = build_vocab_from_iterator(map(tokenizer, texts), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

# Define a function to numericalize the text
def numericalize_text(text):
    return [vocab[token] for token in tokenizer(text)]

<!-- #region id="UuCHfF_CaUId" -->

The `build_vocab_from_iterator()` allows us to build a vocabulary from
an iterator, which can be useful when working with large text corpus.

`map(tokenizer, texts)` builds a vocabulary from an iterator that
applies the tokenizer function to each text in the texts list that we
created earlier. The `specials=['<unk>']` argument specifies that the
special token `<unk>` should be included in the vocabulary, and
`vocab.set_default_index(vocab['<unk>']` sets the default index of the
vocabulary object vocab to the index of the `<unk>` token. This means
that any token that is not present in the vocabulary will be replaced
with the `<unk>` token during numericalization.

We’ll have some checks for the constructed vocabulary for our text set
and word indexes below: <!-- #endregion -->

In [ ]:
# the length of the constructed vocab from the text set, 100683 unique tokens
print(len(vocab))

2749


In [ ]:
# checking the index of words that are present in the vocabulary
print(vocab(['este', 'es', 'un', 'ejemplo','en','español','.']))

[233, 13, 21, 0, 5, 0, 3]


In [ ]:
# Al comprobar el índice de una palabra que no está presente en el vocabulario, devuelve 0, el índice de <unk>
print(vocab['estabilidad'])

0


Now, let’s prepare the training and validation dataset that we will be
using in our model. Using PyTorch’s Dataset and Dataloader functionality
is an efficient way to manage your data and simplify your machine
learning workflow. By creating a Dataset, you can easily store all of
your data in a structured manner. Dataloader, on the other hand, allows
you to iterate through the data, manage batches, and perform data
transformations, among other tasks. Together, these tools can help make
your data more manageable and streamline your machine learning pipeline.

We will build a custom text dataset from the texts and labels we created
earlier, to learn the basics of how to build a custom text dataset,
follow the tutorial in [this
link](https://towardsdatascience.com/how-to-use-datasets-and-dataloader-in-pytorch-for-custom-text-data-270eed7f7c00).

We can numericalize the text in the train dataset in the same custom
dataset classs by the help of `numericalize_text()` that we defined in
the earlier step, we can modify the `__getitem__` method of the
CustomTextDataset class to apply the `numericalize_text()` function to
each text sample, as follows: <!-- #endregion -->

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split

#  Definir una clase de conjunto de datos personalizada para los datos de texto
class CustomTextDataset(Dataset):
    def __init__(self, texts, labels, vocab, numericalize_text):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.numericalize_text = numericalize_text

    def __getitem__(self, index):
        label = self.labels[index]
        text = self.texts[index]
        numericalized_text = self.numericalize_text(text)
        return numericalized_text, label

    def __len__(self):
        return len(self.labels)

In the code above, we modified the `__getitem__` method to first extract
the text and label from the dataset, then apply the numericalize_text
function to the text sample to get a list of numericalized tokens. The
numericalized tokens are then stored in the “Text” field of the sample
dictionary along with the label, which is stored in the “Class” field.
Finally, the sample dictionary is returned.

We also added the vocab and tokenizer arguments to the CustomTextDataset
constructor so that the tokenizer and vocabulary can be passed to the
dataset. This allows the numericalize_text function to use the
vocabulary and tokenizer defined earlier in the code.


Let’s create the dataset now as an object of CustomeTextDataset class,
and create validation set by setting apart 20% of the training dataset.

In [ ]:
# Create train and validation datasets
dataset = CustomTextDataset(texts, labels, vocab, numericalize_text)
train_size = int(len(dataset) * 0.8)
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

Now, we’ll generate the data batch. To generate the data batch,
`torch.utils.data.DataLoader` is used here.

Before sending to the model, `collate_fn` function works on a batch of
samples generated from `DataLoader`. The input to `collate_fn` is a
batch of data with the batch size in `DataLoader`, and `collate_fn`
processes them according to the data processing a custom pipeline that
we’ll define in the following function named `collate_batch`, as
follows.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torch.nn.utils.rnn import pad_sequence

# preprocess the data with a collate function, and pads the input sequences to the maximum length in the batch:
def collate_batch(batch):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label + 1)
        processed_text = torch.tensor(_text)
        text_list.append(processed_text)

    # No es necesario verificar el rango de las etiquetas aquí

    padded_text = pad_sequence(text_list, batch_first=False, padding_value=1.0)
    return torch.tensor(label_list, dtype=torch.long).to(device), padded_text.to(device)

# Create train and validation data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, collate_fn=collate_batch, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, collate_fn=collate_batch, batch_size=batch_size, shuffle=False)

You can see the benefit of the custom defined `collate_batch()` function
we defined above, we used it, for instance, to pad the sequences with
`padding_value=1.0`, the benefit of padding the sequence with 1.0 in the
previous step is to make all the input sequences have the same length,
which is required by most deep learning models for NLP. Padding also
helps to preserve the information at the beginning and end of the
sequence, which can be important for some tasks. Padding with 1.0 is a
common choice because it is a neutral value that does not interfere with
other tokens.

We need such a step because natural language sentences have variable
lengths, but deep learning models expect fixed-size inputs. For example,
if we want to use a recurrent neural network (RNN) to process a batch of
sentences, we need to pad them to the maximum length in the batch so
that they can be fed into the deep learning model as a matrix.

An example of padding is shown below:

Original sentences:
`[“I really love this movie”, “It was terrible”, “The acting was good”]`

Tokenized sentences:
`[[9, 14, 56, 11, 17], [10, 21, 88], [8, 45, 21, 32]]`

Padded sentences to the longest sentence in our list (with maxlen=5):
`[[9, 14, 56, 11, 17], [10, 21, 88 ,1 ,1], [8 ,45 ,21 ,32 ,1]]`

Let’s make a last check to see how the data is stored in our
`train_loader` batches:

In [ ]:
label, text = next(iter(train_loader))
print(label.shape, text.shape)
print(label, text)

torch.Size([32]) torch.Size([176, 32])
tensor([3, 4, 1, 1, 2, 2, 3, 4, 2, 1, 3, 3, 4, 4, 2, 2, 3, 3, 2, 1, 4, 3, 1, 4,
        3, 1, 3, 1, 1, 2, 1, 3]) tensor([[ 62,  18,  39,  ...,  29, 120,  39],
        [  7, 110, 198,  ..., 212, 319, 321],
        [485,  27,   9,  ...,   5,  45, 339],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]])


One last note before starting the modelling, we used `batch_first=False`
in `pad_sequence` in our custom collate function above, and the benefit
is to make the padded sequences have a shape of (sequence_length,
batch_size), which is the expected input shape for the model in PyTorch.

Great, we are done with preprocessing our data, let’s build our model
now.

#### Model Design

<!-- #endregion -->
<!-- #region id="wkEEsz7jzIJG" -->

We’ll create a simple model for text classification. The model is
composed of an `nn.Embedding` plus a linear layer for the classication
purpose. <!-- #endregion -->

<!-- #region id="Al-KQSHw1zpg" -->

<strong>But, what is the embedding layer, why did we use it, and how
does it work?</strong>

We could feed the list of tokenized sequences e,g.
`[8, 124, 3732,  ..., 1197, 71, 4635],[145, 2402, 1,  ..., 74, 57, 4],         ...,`
directly to our model classifier layer, but our model won’t make sense
of the meaning of the words in our sentences or the relationship between
the words in the sequence. The technique that we will be using here to
do so is called the word embeddings with the help of `Embedding` layer
from `torch.nn`.

Word embedding is a technique where individual words are represented as
real-valued vectors in a lower-dimensional space and captures inter-word
semantics. Word embeddings can be used to store word meanings and
retrieve them using indices, as well as to measure the similarity or
distance between words based on their vectors.

The benefit of word embeddings is that they can preserve syntactic and
semantic information of words and reduce the dimensionality of text data
compared to other methods such as one-hot encoding or bag-of-words.

To learn more about the advantages of word embeddings compared to other
methods, see [this
link](https://www.turing.com/kb/guide-on-word-embeddings-in-nlp).

One way to best learn the embeddings of the words is to use it along
with our text classification task. The `nn.Embedding` layer makes this
possible, and the backpropagation technique of model trainining will
learn the best weights of the layer along with our task.

The output of the nn.embedding layer in the example is a tensor of shape
(sequence_length, batch_size, embedding_dim) that contains the
embeddings for each token in the input text. It’s common to see word
embeddings that are 256-dimensional, 512-dimensional, or
1024-dimensional when dealing with very large vocabularies. In our case
we’ll set the output dimension as 100.

Hence, our model defintion will be as follows:

In [ ]:
from torch import nn
import torch.nn.functional as F

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.permute(1, 0, 2)
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1)
        return self.fc(pooled)

Our model is a simple text classification model that consists of two
layers: an embedding layer and a linear layer. It first applies the
embedding layer to get a low-dimensional representation of each token in
the text. Then it permutes the dimensions of the embedded tensor to
match the expected input shape of the average pooling layer. Next, it
applies the average pooling layer to get a fixed-length representation
of the whole text by averaging over all tokens. Finally, it passes this
representation to the linear layer to get a single output value for each
text.

The structure and layers of the model are shown below:

Input text -\> Embedding layer -\> Average pooling -\> Linear layer -\>
Output value

In [ ]:
#  Cree una instancia del modelo de clasificación de texto con el tamaño de vocabulario especificado, la dimensión de incrustación y la dimensión de salida

model = TextClassificationModel(vocab_size=len(vocab), embedding_dim=100, output_dim=4)

In [ ]:
# Define a loss function based on binary cross entropy and sigmoid activation
criterion = nn.CrossEntropyLoss()
# Define an optimizer that updates the model parameters using Adam algorithm
model = TextClassificationModel(vocab_size=len(vocab), embedding_dim=100, output_dim=4)
optimizer = torch.optim.Adam(model.parameters())

# Move the model to the device (CPU or GPU) for computation
model = model.to(device)

In [ ]:
# En el bucle de entrenamiento
for epoch in range(20):
    epoch_loss = 0
    epoch_acc = 0

    model.train()
    for label, text in train_loader:
        optimizer.zero_grad()
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, label - 1)  # Ajusta las etiquetas en la pérdida

        _, predicted = torch.max(predictions, 1)
        correct = (predicted + 1 == label).float()  # Ajusta la comparación con las etiquetas

        acc = correct.sum() / len(correct)

        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    print("Epoch %d Train: Loss: %.4f Acc: %.4f" % (epoch + 1, epoch_loss / len(train_loader),
                                                    epoch_acc / len(train_loader)))
    epoch_loss = 0
    epoch_acc = 0
    model.eval()

    with torch.no_grad():
        for label, text in val_loader:
            predictions = model(text).squeeze(1)
            loss = criterion(predictions, label - 1)  # Ajusta las etiquetas en la pérdida

            # Corrige el cálculo de la precisión
            _, predicted = torch.max(predictions, 1)
            correct = (predicted + 1 == label).float()  # Ajusta la comparación con las etiquetas

            acc = correct.sum() / len(correct)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    print("Epoch %d Valid: Loss: %.4f Acc: %.4f" % (epoch + 1, epoch_loss / len(val_loader),
                                                    epoch_acc / len(val_loader)))

Epoch 1 Train: Loss: 1.3822 Acc: 0.2813
Epoch 1 Valid: Loss: 1.3587 Acc: 0.3592
Epoch 2 Train: Loss: 1.3512 Acc: 0.3872
Epoch 2 Valid: Loss: 1.3267 Acc: 0.4443
Epoch 3 Train: Loss: 1.3135 Acc: 0.4839
Epoch 3 Valid: Loss: 1.2896 Acc: 0.5384
Epoch 4 Train: Loss: 1.2721 Acc: 0.5381
Epoch 4 Valid: Loss: 1.2446 Acc: 0.4798
Epoch 5 Train: Loss: 1.2140 Acc: 0.5946
Epoch 5 Valid: Loss: 1.1856 Acc: 0.6035
Epoch 6 Train: Loss: 1.1545 Acc: 0.6603
Epoch 6 Valid: Loss: 1.1232 Acc: 0.6856
Epoch 7 Train: Loss: 1.0913 Acc: 0.6802
Epoch 7 Valid: Loss: 1.0672 Acc: 0.6831
Epoch 8 Train: Loss: 1.0240 Acc: 0.7066
Epoch 8 Valid: Loss: 1.0062 Acc: 0.7153
Epoch 9 Train: Loss: 0.9662 Acc: 0.7358
Epoch 9 Valid: Loss: 0.9600 Acc: 0.6876
Epoch 10 Train: Loss: 0.9077 Acc: 0.7723
Epoch 10 Valid: Loss: 0.9092 Acc: 0.7367
Epoch 11 Train: Loss: 0.8569 Acc: 0.7797
Epoch 11 Valid: Loss: 0.8707 Acc: 0.7099
Epoch 12 Train: Loss: 0.8106 Acc: 0.7979
Epoch 12 Valid: Loss: 0.8292 Acc: 0.7352
Epoch 13 Train: Loss: 0.7671 Acc: 

Train the model for 10 epochs, print the training and validation loss
and accuracy for each epoch:

After 10 epochs of training, we get around 89% accuracy on the
validation dataset (note, your result may vary). This is still a simple
model, the code can be modified or extended by changing some parameters
or adding more layers.

Let’s see how it performs on the test dataset, but let’s first prepare
it as we did for the training and validation datasets before.

In [ ]:
# Read the text data and labels from the test directory
test_labels, test_texts = read_text_data(data_path/'test')

# Create a custom text dataset object for the test data using the vocabulary and numericalize function
test_dataset = CustomTextDataset(test_labels, test_texts, vocab, numericalize_text)

# Create a data loader for the test dataset
test_loader = DataLoader(test_dataset, collate_fn=collate_batch, batch_size=batch_size, shuffle=False)

In [ ]:
test_loss = 0
test_acc = 0
model.eval()

with torch.no_grad():
    for label, text in test_loader:
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, label - 1)  # Ajusta las etiquetas en la pérdida

        # Ajusta la comparación de las etiquetas en la evaluación
        # Corrige el cálculo de la precisión
        _, predicted = torch.max(predictions, 1)
        correct = (predicted + 1 == label).float()  # Ajusta la comparación con las etiquetas

        acc = correct.sum() / len(correct)

        test_loss += loss.item()
        test_acc += acc.item()

print("Test: Loss: %.4f Acc: %.4f" %
      (test_loss / len(test_loader),
       test_acc / len(test_loader)))

Test: Loss: 1.0124 Acc: 0.6600


In [ ]:
label_map = {'sis': 0, 'dis': 1, 'cic': 2, 'tis': 3}

In [ ]:

# Tokeniza la frase
#tokenized_text = numericalize_text("Me gusta resolver ejercicios de seguridad.")
tokenized_text = numericalize_text("Me apasiona la creatividad y quiero canalizar mi expresión artística a través del diseño y la animación digital.")
# tokenized_text = numericalize_text("Estoy ansioso por aprender sobre ciberseguridad y contribuir a la protección de la información digital.")
# tokenized_text = numericalize_text("Quiero ser parte del diseño de sistemas de detección de intrusiones que protejan activamente las redes.")
# Convierte la lista de tokens a un tensor y agrega una dimensión adicional
input_text = torch.tensor(tokenized_text).unsqueeze(1).to(device)

# Pasa la entrada a través del modelo
with torch.no_grad():
    model.eval()
    output = model(input_text)

# Obtiene las predicciones
_, predicted_label = torch.max(output, 1)

# Imprime la etiqueta predicha
print("Etiqueta predicha:", predicted_label.item()+1)
predicted_category = list(label_map.keys())[predicted_label.item()]

# Imprime la categoría predicha
print("Categoría predicha:", predicted_category)

Etiqueta predicha: 2
Categoría predicha: dis


We get around 88% accuracy on the test dataset. Not bad!

Let’s see how it works on some random input text data, we’ll do some
housekeeping jobs first: <!-- #endregion -->

Well done! Our model gives a very low score for a negative sentiment and
gives 1 for a positive movie review.

Our model is doing great, let’s save it for inference later on:

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/IA22023/DATASETS/ticsclassification-model.pt')

In [ ]:
loaded_model = TextClassificationModel(vocab_size=len(vocab), embedding_dim=100, output_dim=4) # Crea una nueva instancia del modelo

# Cargamos los pesos del modelo guardado
loaded_model.load_state_dict(torch.load('/content/drive/MyDrive/IA22023/DATASETS/ticsclassification-model.pt'))
loaded_model = loaded_model.to(device)
loaded_model.eval()


new_tokenized_text = numericalize_text("Me gusta resolver ejercicios de lógica.") # Tokeniza la nueva frase

# Convierte la lista de tokens a un tensor y agrega una dimensión adicional
new_input_text = torch.tensor(new_tokenized_text).unsqueeze(1).to(device)

# Pasa la entrada a través del modelo cargado
with torch.no_grad():
    output = loaded_model(new_input_text)

# Obtiene las predicciones
_, predicted_label = torch.max(output, 1)

# Imprime la etiqueta predicha
print("Etiqueta predicha:", predicted_label.item() + 1)

# Mapea las etiquetas predichas a las categorías correspondientes
predicted_category = list(label_map.keys())[predicted_label.item()]

# Imprime la categoría predicha
print("Categoría predicha:", predicted_category)

Etiqueta predicha: 3
Categoría predicha: cic
